In [1]:
from pythreejs import *
import ipywidgets
from IPython.display import display
from pathlib import Path

In [2]:
np.set_printoptions(precision=2)

In [3]:
# simulation params

#time step 
dt = 1.0
#no. of rows
rows = 5
#no. of cols
cols = 5
#stretch constant
ks = 5e5 #5e10
#shear constant
ksh = 100.0 #100.0
#damping constant
kd = 10
#no. of particles 
N = rows*cols
#cloth desity
density = 10.0
#gravity
g = 0.981

# max stretch in u and v direction
bu = 1.0
bv = 1.0

In [4]:
# instantiate particle positions, velocities and forces
# positions
points = []
for r in range(rows+1):
    for c in range(cols+1):
        points.append(r)
        points.append(c)
        points.append(0.0)

# 3D world positions of the particles
world_pos = np.transpose(np.array([points]))  
# positions in the UV space
uv_pos = np.transpose(np.array([points]))

# velocity
vel = np.array([[0.0] for i in range(3*(rows+1)*(cols+1))])

# force
force = np.array([[0.0] for i in range(3*(rows+1)*(cols+1))])
# external force - add gravity in the z direction
for i in range(2, force.shape[0], 3):
    force[i,0] = -g      


# force jacobian matrix
A = np.zeros((3,3))
J = np.block([[A for i in range((rows+1)*(cols+1))] for j in range((rows+1)*(cols+1))])

In [5]:
# instantiate inverse mass matrix
invM = np.array([[0.0 for i in range(3*(rows+1)*(cols+1))] for i in range(3*(rows+1)*(cols+1))])

# mass matrix
for i in range(0, world_pos.shape[0], 3):
    x = world_pos[i,0]
    y = world_pos[i+1,0]
    z = world_pos[i+2,0]
    
    u = uv_pos[i,0]
    v = uv_pos[i+1,0]
    t = uv_pos[i+2,0]
    
    area_of_tri = 0.5
    
    # case 1 : point is inside the grid and has 6 neighbours(belongs to 6 triangles)
    if ((1 <= u <= rows-1) and (1 <= v <= cols-1)):
        tri_mass = density * area_of_tri #(area of triangle is 0.5)
        invM[i,i] = 1.0/((1/3)*6*tri_mass)
        invM[i+1, i+1] = 1.0/((1/3)*6*tri_mass)
        invM[i+2, i+2] = 1.0/((1/3)*6*tri_mass)
#         if ((4 <= u <= 6) and (4 <= v <= 6)):
#         invM[i,i] = 0.0
#         invM[i+1, i+1] = 0.0
#         invM[i+2, i+2] = 0.0
    # case 2 : point is in the corner of the grid and has 1 neighbour 
    elif ((u==0 and v==0) or (u==0 and v==rows) or (u==cols and v==0) or(u==cols and v==rows)):
        tri_mass = density * area_of_tri #(area of triangle is 0.5)
#         invM[i,i] = 1.0/((1/3)*tri_mass)
#         invM[i+1, i+1] = 1.0/((1/3)*tri_mass)
#         invM[i+2, i+2] = 1.0/((1/3)*tri_mass)
#         if ( v==rows) :
#             print(u,v)
        invM[i,i] = 0.0
        invM[i+1, i+1] = 0.0
        invM[i+2, i+2] = 0.0
    # case 3 : point is on the edge of the grid and has 3 neighbours
    else :
        tri_mass = density * area_of_tri #(area of triangle is 0.5)
#         invM[i,i] = 1.0/((1/3)*3*tri_mass)
#         invM[i+1, i+1] = 1.0/((1/3)*3*tri_mass)
#         invM[i+2, i+2] = 1.0/((1/3)*3*tri_mass)
#         if (v == rows):
#             print(u,v)
        invM[i,i] = 0.0
        invM[i+1, i+1] = 0.0
        invM[i+2, i+2] = 0.0


In [6]:
# stretch force
# k - stretch coeff
# alpha - area of the triangle in uv space
def stretch_force_on_all_particles_in_a_triangle(uv_pos, world_pos, idx1, idx2, idx3, k, bu, bv, alpha):
    uv_i = uv_pos[idx1]
    uv_j = uv_pos[idx2] 
    uv_k = uv_pos[idx3] 
    
    p_i = world_pos[idx1] 
    p_j = world_pos[idx2] 
    p_k = world_pos[idx3]
    
    u_i = uv_i[0] #xz
    v_i = uv_i[1]
    
    u_j = uv_j[0] #xz
    v_j = uv_j[1]
    
    u_k = uv_k[0] #xz
    v_k = uv_k[1]
    
    dv1 = v_j - v_i
    dv2 = v_k - v_i
    du1 = u_j - u_i
    du2 = u_k - u_i
    
    x_i = p_i[0]
    y_i = p_i[1]
    z_i = p_i[2]
    X_i = np.array([[p_i[0]], [p_i[1]], [p_i[2]]])
    
    x_j = p_j[0]
    y_j = p_j[1]
    z_j = p_j[2]
    X_j = np.array([[p_j[0]], [p_j[1]], [p_j[2]]])
    
    x_k = p_k[0]
    y_k = p_k[1]
    z_k = p_k[2]
    X_k = np.array([[p_k[0]], [p_k[1]], [p_k[2]]])
    
    # calculate wu, wv
    Wu = ((X_j - X_i)*dv2 - (X_k - X_i)* dv1) / (du1*dv2 - du2*dv1)
    Wu = Wu / np.linalg.norm(Wu)
    
    Wv = (-(X_j - X_i)*du2 + (X_k - X_i)* du1) / (du1*dv2 - du2*dv1)
    Wv = Wv / np.linalg.norm(Wv)
    
    # condition function
    Cu = np.linalg.norm(Wu) - bu
    Cv = np.linalg.norm(Wv) - bv
    
    C = np.array([[Cu], [Cv]])
    C_T = np.transpose(C)
    
    # calculate dwu/dx, dwu/dx 
    # for Xi
    dWu_dXi = (dv1 - dv2) / (du1*dv2 - du2*dv1)
    dWv_dXi = (du2 - du1) / (du1*dv2 - du2*dv1)
    
    # for Xj
    dWu_dXj = dv2 / (du1*dv2 - du2*dv1)
    dWv_dXj = -du2 / (du1*dv2 - du2*dv1)
    
    # for Xk
    dWu_dXk = -dv1 / (du1*dv2 - du2*dv1)
    dWv_dXk = du1 / (du1*dv2 - du2*dv1)
    
    # force on Xi
    f_i_u = (-k * alpha * dWu_dXi * np.identity(3, dtype = float).dot(Wu)) * Cu
    f_i_v = (-k * alpha * dWv_dXi * np.identity(3, dtype = float).dot(Wv)) * Cv
    f_i = f_i_u + f_i_v
    
    # force on Xj
    f_j_u = (-k * alpha * dWu_dXj * np.identity(3, dtype = float).dot(Wu)) * Cu
    f_j_v = (-k * alpha * dWv_dXj * np.identity(3, dtype = float).dot(Wv)) * Cv
    f_j = f_j_u + f_j_v
    
    # force on Xk
    f_k_u = (-k * alpha * dWu_dXk * np.identity(3, dtype = float).dot(Wu)) * Cu
    f_k_v = (-k * alpha * dWv_dXk * np.identity(3, dtype = float).dot(Wv)) * Cv
    f_k = f_k_u + f_k_v
    
    # calculate jacobian matrix
    # u direction
    temp_mat_1 = (dWu_dXi * np.identity(3, dtype = float)).dot(dWu_dXj * np.identity(3, dtype = float))

    temp_mat_2 = np.identity(3, dtype=float) - Wu.dot(np.transpose(Wu))
                                                                        
    Kij_u = -k * (alpha * dWu_dXi * np.identity(3, dtype = float).dot(Wu) * \
                alpha * np.transpose(dWu_dXj * (np.identity(3, dtype = float).dot(Wu))) + \
                (alpha / np.linalg.norm(Wu)) * temp_mat_1.dot(temp_mat_2) * Cu)
                                                                
    temp_mat_3 = (dWu_dXj * np.identity(3, dtype = float)).dot(dWu_dXk * np.identity(3, dtype = float))
                                                                        
    Kjk_u = -k * (alpha * dWu_dXj * np.identity(3, dtype = float).dot(Wu) * \
                alpha * np.transpose(dWu_dXk * (np.identity(3, dtype = float).dot(Wu))) + \
                (alpha / np.linalg.norm(Wu)) * temp_mat_3.dot(temp_mat_2) * Cu)
                                                                
    temp_mat_4 = (dWu_dXk * np.identity(3, dtype = float)).dot(dWu_dXi * np.identity(3, dtype = float))
                                                                        
    Kki_u = -k * (alpha * dWu_dXk * np.identity(3, dtype = float).dot(Wu) * \
                alpha * np.transpose(dWu_dXi * (np.identity(3, dtype = float).dot(Wu))) + \
                (alpha / np.linalg.norm(Wu)) * temp_mat_4.dot(temp_mat_2) * Cu)
              
    # v direction                                                            
    temp_mat_1 = (dWv_dXi * np.identity(3, dtype = float)).dot(dWv_dXj * np.identity(3, dtype = float))
    temp_mat_2 = np.identity(3, dtype=float) - Wv.dot(np.transpose(Wv))
                                                                        
    Kij_v = -k * (alpha * dWv_dXi * np.identity(3, dtype = float).dot(Wv) * \
                alpha * np.transpose(dWv_dXj * (np.identity(3, dtype = float).dot(Wv))) + \
                (alpha / np.linalg.norm(Wv)) * temp_mat_1.dot(temp_mat_2) * Cv)
                                                                
    temp_mat_3 = (dWv_dXj * np.identity(3, dtype = float)).dot(dWv_dXk * np.identity(3, dtype = float))
                                                                        
    Kjk_v = -k * (alpha * dWv_dXj * np.identity(3, dtype = float).dot(Wv) * \
                alpha * np.transpose(dWv_dXk * (np.identity(3, dtype = float).dot(Wv))) + \
                (alpha / np.linalg.norm(Wv)) * temp_mat_3.dot(temp_mat_2) * Cv)
                                                                
    temp_mat_4 = (dWv_dXk * np.identity(3, dtype = float)).dot(dWv_dXi * np.identity(3, dtype = float))
                                                                         
    Kki_v = -k * (alpha * dWv_dXk * np.identity(3, dtype = float).dot(Wv) * \
                alpha * np.transpose(dWv_dXi * (np.identity(3, dtype = float).dot(Wv))) + \
                (alpha / np.linalg.norm(Wv)) * temp_mat_4.dot(temp_mat_2) * Cv)
   
    Kij = Kij_u + Kij_v
    Kjk = Kjk_u + Kjk_v
    Kki = Kki_u + Kki_v
                                                                
    return f_i, f_j, f_k, Kij, Kjk, Kki

In [7]:
# shear force
# k - shear coeff
# alpha - area of the triangle in uv space
def shear_force_on_all_particles_in_a_triangle(uv_pos, world_pos, idx1, idx2, idx3, k, bu, bv, alpha):
    uv_i = uv_pos[idx1]
    uv_j = uv_pos[idx2] 
    uv_k = uv_pos[idx3] 
    
    p_i = world_pos[idx1] 
    p_j = world_pos[idx2] 
    p_k = world_pos[idx3]
    
    u_i = uv_i[0] #xz
    v_i = uv_i[1]
    
    u_j = uv_j[0] #xz
    v_j = uv_j[1]
    
    u_k = uv_k[0] #xz
    v_k = uv_k[1]
    
    dv1 = v_j - v_i
    dv2 = v_k - v_i
    du1 = u_j - u_i
    du2 = u_k - u_i
    # print("the deltas : ",dv1, dv2, du1, du2)

    x_i = p_i[0]
    y_i = p_i[1]
    z_i = p_i[2]
    X_i = np.array([[p_i[0]], [p_i[1]], [p_i[2]]])
    
    x_j = p_j[0]
    y_j = p_j[1]
    z_j = p_j[2]
    X_j = np.array([[p_j[0]], [p_j[1]], [p_j[2]]])
    
    x_k = p_k[0]
    y_k = p_k[1]
    z_k = p_k[2]
    X_k = np.array([[p_k[0]], [p_k[1]], [p_k[2]]])
    
    # calculate wu, wv
    Wu = ((X_j - X_i)*dv2 - (X_k - X_i)* dv1) / (du1*dv2 - du2*dv1)
    Wu = Wu / np.linalg.norm(Wu)
    
    Wv = (-(X_j - X_i)*du2 + (X_k - X_i)* du1) / (du1*dv2 - du2*dv1)
    Wv = Wv / np.linalg.norm(Wv)
    
    # condition function
    C = alpha * (np.transpose(Wu)).dot(Wv)
    
    # calculate dwu/dx, dwu/dx 
    # for Xi
    dWu_dXi = (dv1 - dv2) / (du1*dv2 - du2*dv1)
    dWv_dXi = (du2 - du1) / (du1*dv2 - du2*dv1)
    
    # for Xj
    dWu_dXj = dv2 / (du1*dv2 - du2*dv1)
    dWv_dXj = -du2 / (du1*dv2 - du2*dv1)
    
    # for Xk
    dWu_dXk = -dv1 / (du1*dv2 - du2*dv1)
    dWv_dXk = du1 / (du1*dv2 - du2*dv1)
    
    dC_dXi = alpha * (dWu_dXi * np.identity(3, dtype = float).dot(Wv) + dWv_dXi * np.identity(3, dtype = float).dot(Wu)) 
    dC_dXj = alpha * (dWu_dXj * np.identity(3, dtype = float).dot(Wv) + dWv_dXj * np.identity(3, dtype = float).dot(Wu)) 
    dC_dXk = alpha * (dWu_dXk * np.identity(3, dtype = float).dot(Wv) + dWv_dXk * np.identity(3, dtype = float).dot(Wu)) 
    
    f_i = -k * dC_dXi * C
    f_j = -k * dC_dXj * C
    f_k = -k * dC_dXk * C
    
    # calculate the jacobian matrix
    K_ij = -k* (dC_dXi.dot(np.transpose(dC_dXj)) + alpha*(dWu_dXi*dWv_dXj+dWu_dXj*dWv_dXi)*np.identity(3, dtype=float)*C )
    
    K_jk = -k* (dC_dXj.dot(np.transpose(dC_dXk)) + alpha*(dWu_dXj*dWv_dXk+dWu_dXk*dWv_dXj)*np.identity(3, dtype=float)*C ) 
    
    K_ki = -k* (dC_dXk.dot(np.transpose(dC_dXi)) + alpha*(dWu_dXk*dWv_dXi+dWu_dXi*dWv_dXk)*np.identity(3, dtype=float)*C  )
    
    return f_i, f_j, f_k, K_ij, K_jk, K_ki

In [8]:
# Given an index, return list of indices coming from the neighbouring traingles. 
def get_neighbouring_triangles(index, u, v, gridX, gridY):
    triangle_indices = []
    rows = gridX - 1
    cols = gridY - 1
    
    # case 1 : point is inside the grid and has 6 neighbours(belongs to 6 triangles)
    if ((1 <= u <= rows-1) and (1 <= v <= cols-1)):
        # particle indices
            tri_1 = [index, index+1, index+1+gridX ]
            tri_2 = [index, index+1+gridX, index+gridX ]
            tri_3 = [index, index+gridX, index-1 ]
            tri_4 = [index, index-1, index-1-gridX ]
            tri_5 = [index-1-gridX, index-gridX, index ]
            tri_6 = [index, index-gridX, index+1 ]
            triangle_indices = [tri_1, tri_2, tri_3, tri_4, tri_5, tri_6]
    # case 2 : point is in the corner of the grid and has 1 neighbour 
    elif ((u==0 and v==0) or (u==0 and v==rows) or (u==cols and v==0) or(u==cols and v==rows)):
        if (u==0 and v==0):
            triangle_indices = [[index, index+1, index+gridX+1], 
                                [index, index+1+gridX, index+gridX]]
        if (u==0 and v==cols):
            triangle_indices = [[index, index-1, index+gridX]]
        if (u==cols and v==rows):
            triangle_indices = [[index, index-1, index-gridX-1], 
                                [index, index-gridX-1, index-gridX]]
        if (u==rows and v==0):
            triangle_indices = [[index, index+1, index-gridX]]
    # case 3 : point is on the edge of the grid and has 3 neighbours
    else :
        if (v == 0) :
            triangle_indices = [[index, index-gridX, index+1], 
                                [index, index+1, index+1+gridX], 
                                [index, index+1+gridX, index+gridX]]
        if (v == cols):
            triangle_indices = [[index, index+gridX, index-1], 
                                [index, index-1, index-1-gridX], 
                                [index, index-1-gridX, index-gridX]]
        if (u == rows):
            triangle_indices = [[index, index+1, index-gridX], 
                                [index, index - gridX, index-1-gridX], 
                                [index, index-1-gridX, index-1]]
        if (u == 0):
            triangle_indices = [[index, index-1, index+gridX], 
                                [index, index + gridX, index+1+gridX], 
                                [index, index+1+gridX, index+1]]
    
    
    return triangle_indices


In [9]:
# input dimensions
# world_pos - 300X1
# uv_pos - 300X1
# f - 300X1
# df_dx - 300X300
def update_force_and_jacobian(world_pos, rows, cols, uv_pos, ks, bu, bv, f, J):
    gridX = cols + 1
    gridY = rows + 1
    world_pos_reshaped = world_pos.reshape(gridX*gridY, 3)
    uv_pos_reshaped = uv_pos.reshape(gridX*gridY, 3)
    f_reshaped = f.reshape(gridX*gridY, 3)
    
    for index in range(world_pos_reshaped.shape[0]):
        x = world_pos_reshaped[index][0]
        y = world_pos_reshaped[index][1]
        z = world_pos_reshaped[index][2]
        
        u = uv_pos_reshaped[index][0]
        v = uv_pos_reshaped[index][1]
        t = uv_pos_reshaped[index][2]
        alpha = 0.5
        
        triangle_indices = get_neighbouring_triangles(index, u, v, gridX, gridY)
        

        for triangle in triangle_indices:
            f_p_1_st, f_p_2_st, f_p_3_st, Kij_st, Kjk_st, Kki_st = stretch_force_on_all_particles_in_a_triangle(
                                                                    uv_pos_reshaped, world_pos_reshaped, 
                                                                    triangle[0], triangle[1], triangle[2],
                                                                    ks, bu, bv, alpha)
                
            f_p_1_sh, f_p_2_sh, f_p_3_sh, Kij_sh, Kjk_sh, Kki_sh = shear_force_on_all_particles_in_a_triangle(
                                                                    uv_pos_reshaped, world_pos_reshaped, 
                                                                    triangle[0], triangle[1], triangle[2],
                                                                    ksh, bu, bv, alpha)
                
                
            f_p_1 = f_p_1_st + f_p_1_sh
            f_p_2 = f_p_2_st + f_p_2_sh
            f_p_3 = f_p_3_st + f_p_3_sh
                
            # print("shear force - ", f_p_1_sh, f_p_2_sh, f_p_3_sh)
            # print("stretch force - ", f_p_1_st, f_p_2_st, f_p_3_st)
                
            Kij = Kij_st + Kij_sh 
            Kjk = Kjk_st + Kjk_sh
            Kki = Kki_st + Kki_sh
                

                
            f_reshaped[triangle[0]] += f_p_1.reshape(1,3)[0]
            f_reshaped[triangle[1]] += f_p_2.reshape(1,3)[0]
            f_reshaped[triangle[2]] += f_p_3.reshape(1,3)[0]

            #Kij
            J[3*triangle[0]:3*triangle[0]+3, 3*triangle[1]:3*triangle[1]+3] += Kij
            #Kji
            J[3*triangle[1]:3*triangle[1]+3, 3*triangle[0]:3*triangle[0]+3] += Kij
            #Kjk
            J[3*triangle[1]:3*triangle[1]+3, 3*triangle[2]:3*triangle[2]+3] += Kjk
            #Kkj
            J[3*triangle[2]:3*triangle[2]+3, 3*triangle[1]:3*triangle[1]+3] += Kjk
            #Kki
            J[3*triangle[2]:3*triangle[2]+3, 3*triangle[0]:3*triangle[0]+3] += Kki
            #Kik
            J[3*triangle[0]:3*triangle[0]+3, 3*triangle[2]:3*triangle[2]+3] += Kki

    new_f = f_reshaped.reshape(gridX*gridY*3, 1) 
    return new_f, J
           

In [10]:
# be - backward euler
# dimensions
# pos - 300X1
# vel - 300X1
# force - 300X1
# invM - 300X300
def be(world_pos, uv_pos, rows, cols, vel, force, jacobian, ks, bu, bv, invM, dt):
    
#     for every pos :
#         calculate net force
#         calculate force jacobian
#         calculate     
#             Adv = b
#             A = I - dt^2*Minv*df_dx
#             b = dt*Minv*(f_prev + dt*df_dx*v_prev)
#             dv = A\b
#             apply boundary conditions (If applicable)
#             x_new - x_old = dt*(v_prev + dv)
#             update positions
    J = jacobian
    f, df_dx = update_force_and_jacobian(world_pos, rows, cols, uv_pos, ks, bu, bv, force, J)
    
    I = np.identity(3*(rows+1)*(cols+1), dtype=float)
    
    A = I - dt*dt*(invM.dot(df_dx))
    
    b = dt*invM.dot(f + dt*df_dx.dot(vel) )
    
    dv = np.linalg.solve(A, b)

    xnew = world_pos + dt*(vel + dv)
    vnew = vel + dv

    # update positions, velocities, (force and jacobians?)

    return xnew, vnew, f, df_dx

In [11]:

keyframes = {}

# time stepping
t = 0
for i in range(15):
    xnew, vnew, f, df_dx = be(world_pos, uv_pos, rows, cols, vel, force, J, ks, bu, bv, invM, dt)
    world_pos = xnew
    t = t + dt
    keyframes[t] = world_pos.reshape((rows+1)*(cols+1), 3)
#     force = f
#     K = df_dx


In [20]:
# render the animation
# input
# bufferGeo -pythreejs bufferGeometry
# keyframes - A dictionary with time and vertex positions at the time
# { "t1" : [v1, v2, v3....],
#   "t2" : [v1, v2, v3....]...
# }
# dimensions of the vertex positions array should be --> no. of vertices X 3

def draw(bufferGeo, keyframes):
    
    position_morph_attrs = []
    time_steps = []
    frame_indices = []
    frame_index = 0
    for key, value in keyframes.items():
        time_steps.append(key)
        position_morph_attrs.append(BufferAttribute(np.array(value, dtype=np.float32), normalized=False))
        frame_indices.append(frame_index)
        frame_index += 1
    
    bufferGeo.morphAttributes = {'position': position_morph_attrs}
    mesh = Mesh(bufferGeo, MeshStandardMaterial(side='DoubleSide', color='red', wireframe=True, morphTargets=True))
    
    # create key frames
    position_track = NumberKeyframeTrack(
            name='.morphTargetInfluences[0]', times=time_steps, values=frame_indices)
    # create animation clip from the morph targets
    position_clip = AnimationClip(tracks=[position_track])
    # create animation action
    position_action = AnimationAction(AnimationMixer(mesh), position_clip, mesh)

    # setup scene
    camera = PerspectiveCamera( position=[5, 3, 5], aspect=600/400)
    scene = Scene(children=[mesh, camera,
                             DirectionalLight(position=[3, 5, 1], intensity=0.6),
                             AmbientLight(intensity=0.5)])
    renderer = Renderer(camera=camera, scene=scene,
                         controls=[OrbitControls(controlling=camera)],
                         width=600, height=400)

    display(renderer, position_action)
    

In [21]:
# form faces from the vertices, make bufferGeometry with faces and vertices 
def get_triangles(index, uvpos, gridX, gridY):
    tri1 = []
    tri2 = []
    u = uvpos[0]
    v = uvpos[1]
    
    if (u < gridX - 1 and v < gridX - 1): 
        quad = [index, index+1, index+ 1 + gridY, index + gridY]
        tri1 = [index, index+1, index+ 1 + gridY]
        tri2 = [index, index+ 1 + gridY, index + gridY]
    
    return tri1, tri2

# calculate faces to feed into the index array in bufferGeo
uv_pos_reshaped = uv_pos.reshape((rows+1)*(cols+1), 3)
world_pos_reshaped = world_pos.reshape((rows+1)*(cols+1), 3)
faces = []
for index in range(uv_pos_reshaped.shape[0]):
    uvpos = uv_pos_reshaped[index]
    tri1, tri2 = get_triangles(index, uvpos, rows+1, cols+1)
    if (tri1 != [] and tri2 != []):
        faces.append(tri1)
        faces.append(tri2)
faces = np.array(faces,  dtype=np.uint16).ravel()

bufferGeo = BufferGeometry(
        attributes={
            'position': BufferAttribute(np.array(world_pos_reshaped, dtype=np.float32), normalized=False),
            'index': BufferAttribute(faces),
        }
    )

# call the draw function 
draw(bufferGeo, keyframes)

drawing
end drawing


Renderer(camera=PerspectiveCamera(aspect=1.5, position=(5.0, 3.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.morphTargetInfluences[0]', times=array([…